In [1]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [2]:
LOCAL_PATH = 'data'
RAW_TRACE = 'Skype_HongKong.pcapng'
TCP_TRACE_FEATURE_FILE = 'Skype_HongKong_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'Skype_HongKong_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'Skype_HongKong_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'Skype_HongKong_udp_flow.csv'
# BUCKET_NAME = '' # replace with your bucket name
# KEY = '' # replace with your object key

In [3]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [4]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Packet feature file already exists.
CPU times: user 6.22 s, sys: 409 ms, total: 6.63 s
Wall time: 7.31 s
Packet feature file already exists.
CPU times: user 327 ms, sys: 129 ms, total: 455 ms
Wall time: 494 ms


In [5]:
def to_csv(trace_df, extract_func, file, features, max_packets_per_flow):
    extract_func(trace_df, 1.0, upsampled=True, max_packets_per_flow=max_packets_per_flow)[features].to_csv(file, index=False)

In [9]:
from python import flow_feature
reload(flow_feature)

FEATURES = ['avg(pkt_len)', 'stddev(pkt_len)', 'fb_ratio', 'inter_arrival_time', 'pkt_count']

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000)
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000)
    else:
        raise


Max pkt per flow->1000: 100%|██████████| 13027/13027 [00:05<00:00, 2280.24it/s]/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

Max pkt per flow->1000:   3%|▎         | 11321/365216 [00:05<01:49, 3227.59it/s]


Max pkt per flow->1000:  14%|█▎        | 49617/365216 [00:16<01:28, 3547.12it/s]


Max pkt per flow->1000:  25%|██▌       | 93126/365216 [00:27<01:00, 4481.42it/s]


Max pkt per flow->1000:  38%|███▊      | 139038/365216 [00:37<00:44, 5073.52it/s]


Max pkt per flow->1000:  53%|█████▎    | 192215/365216 [00:47<00:34, 4969.27it/s]


Max pkt per flow->1000:  66%|██████▌   | 241715/365216 [00:58<00:32, 3855.59it/s]


Max pkt per flow->1000:  80%|███████▉  | 291396/365216 [01:08<00:11, 6280.06it/s]


Max pkt per flow->1000:  96%|█████████▌| 348867/365216 [01:18<00:03, 4842.85it/s]


Max pkt per flow->1000: 100%|██████████| 365216/365216 [01:21<00:00, 4483.42it/s]/Users/shuojiang/Desktop/GA/Trace-Preprocessing/trace-feature-selection/python/flow_feature.py:81: RuntimeWarning: invalid value encountered in double_scalars
  flow['avg(pkt_len)'] = flow['pkt_len'] / flow['pkt_count']
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
